In [24]:
import pandas as pd
df = pd.read_csv("df_news_content.csv")
df = df.dropna()
   
paragraphs = df['text']


In [26]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer


model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/Users/williamyeung/anaconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [27]:
summaries = []
original_paragraphs = []


for paragraph in paragraphs:
    input_text = "summarize: " + paragraph
    inputs = tokenizer.encode_plus(input_text, return_tensors='pt', max_length=512, truncation=True)

    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=300, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    summaries.append(summary)
    original_paragraphs.append(paragraph)


data = {'Original Paragraph': original_paragraphs, 'Summary': summaries}
df = pd.DataFrame(data)


df



                                    Original Paragraph  \
0    Sign up to our free Brexit and beyond email fo...   
1    Something went wrong, please try again later.\...   
2    Ukip has been ordered to fully reveal details ...   
3    Farage got stuck on his bus in Kent because pe...   
4    Sign up for the View from Westminster email fo...   
..                                                 ...   
141  Rachel Maddow reports on a controversy in the ...   
142  Sign up to our free Brexit and beyond email fo...   
143  Sign up to our free Brexit and beyond email fo...   
144  Advertisement\n\nLawson, former chairman of th...   
145  Vote Leave badge: Almost every aspect of Briti...   

                                               Summary  
0    Andrew Grice’s article on how Boris Johnson mi...  
1    under one in 10 young people in Scotland think...  
2    the ruling casts a shadow over the 2016 EU ref...  
3    three men with balaclavas spotted greeted the ...  
4    theresa may s

In [29]:
df.to_csv('summary.csv')